1.  Please visit this link to access the state-of-art DenseNet code for reference - DenseNet - cifar10 notebook link
2.  You need to create a copy of this and "retrain" this model to achieve 90+ test accuracy. 
3.  You cannot use DropOut layers.
4.  You MUST use Image Augmentation Techniques.
5.  You cannot use an already trained model as a beginning points, you have to initilize as your own
6.  You cannot run the program for more than 300 Epochs, and it should be clear from your log, that you have only used 300 Epochs
7.  You cannot use test images for training the model.
8.  You cannot change the general architecture of DenseNet (which means you must use Dense Block, Transition and Output blocks as mentioned in the code)
9.  You are free to change Convolution types (e.g. from 3x3 normal convolution to Depthwise Separable, etc)
10. You cannot have more than 1 Million parameters in total
11. You are free to move the code from Keras to Tensorflow, Pytorch, MXNET etc. 
12. You can use any optimization algorithm you need. 
13. You can checkpoint your model and retrain the model from that checkpoint so that no need of training the model from first if you lost at any epoch while training. You can directly load that model and Train from that epoch. 

In [1]:
import tensorflow
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [4]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

170508288/170498071 [==============================] - 6s 0us/step


In [5]:
#converting to float
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [6]:
X_train.shape

(50000, 32, 32, 3)

In [7]:
X_test.shape

(10000, 32, 32, 3)

In [8]:
#data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen_train = ImageDataGenerator(rescale=1./255,
                             shear_range=0.2,
                             zoom_range=0.2,
                             width_shift_range=0.2,
                             height_shift_range=0.1,
                             horizontal_flip=True)


datagen_test = ImageDataGenerator(rescale=1./255)

In [9]:
# Dense Block
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [20]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 300
l = 12
num_filter = 35
compression = 0.5
dropout_rate = 0.2

In [36]:
#defining the model

input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [37]:
model = Model(inputs=[input], outputs=[output])

#model summary
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_208 (Conv2D)             (None, 32, 32, 35)   945         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_208 (BatchN (None, 32, 32, 35)   140         conv2d_208[0][0]                 
__________________________________________________________________________________________________
activation_208 (Activation)     (None, 32, 32, 35)   0           batch_normalization_208[0][0]    
____________________________________________________________________________________________

In [38]:
#number of layers in the model
print(len(model.layers))

262


In [39]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [40]:
#callback class to monitor validation accuracy
class Monitor(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs['val_accuracy']>0.90):
            self.model.stop_training = True

In [45]:
filepath="model_save/best_model.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, monitor='val_accuracy',  verbose=1, save_best_only=True, mode='max')

monitor = Monitor()

model.fit(datagen_train.flow(X_train, y_train, batch_size=batch_size),
          validation_data=datagen_test.flow(X_test, y_test, batch_size=batch_size),validation_steps = len(X_test) / batch_size,
          steps_per_epoch=len(X_train) / batch_size, epochs=epochs,callbacks=[checkpoint,monitor])

Epoch 1/300
390/390 [==============================] - 61s 141ms/step - loss: 1.6204 - accuracy: 0.4011 - val_loss: 1.8438 - val_accuracy: 0.3532

Epoch 00001: val_accuracy improved from -inf to 0.35320, saving model to model_save/best_model.hdf5
Epoch 2/300
390/390 [==============================] - 55s 140ms/step - loss: 1.2690 - accuracy: 0.5411 - val_loss: 1.4784 - val_accuracy: 0.5341

Epoch 00002: val_accuracy improved from 0.35320 to 0.53410, saving model to model_save/best_model.hdf5
Epoch 3/300
390/390 [==============================] - 54s 139ms/step - loss: 1.0827 - accuracy: 0.6112 - val_loss: 1.4225 - val_accuracy: 0.5692

Epoch 00003: val_accuracy improved from 0.53410 to 0.56920, saving model to model_save/best_model.hdf5
Epoch 4/300
390/390 [==============================] - 54s 139ms/step - loss: 0.9655 - accuracy: 0.6567 - val_loss: 1.5663 - val_accuracy: 0.5750

Epoch 00004: val_accuracy improved from 0.56920 to 0.57500, saving model to model_save/best_model.hdf5
Epo

In [48]:
# Test the model
score = model.evaluate(datagen_test.flow(X_test, y_test, batch_size=batch_size), steps=len(X_test)/batch_size,verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

78/78 [==============================] - 3s 35ms/step - loss: 0.3589 - accuracy: 0.9058
Test loss: 0.3589215576648712
Test accuracy: 0.9057999849319458


In [49]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk
